# 教學目標

- 複習 PTT 文章的爬蟲邏輯
- 熟悉單網站多網頁，透過列表將連結內的文章內容都爬下來

In [1]:
import requests
import re
import json
from urllib.parse import urljoin
from bs4 import BeautifulSoup

# PTT 八卦版網址
PTT_URL = 'https://www.ptt.cc/bbs/Gossiping/index.html'

## 範例重點

這個練習題在 PTT 單頁文章上的程式碼比較多，建議可以先把他寫成一個 function 方便後面程式邏輯的撰寫

In [16]:
def crawl_article(url):
    response = requests.get(url, cookies={'over18': '1'})
    
    # 假設網頁回應不是 200 OK 的話, 我們視為傳送請求失敗
    if response.status_code != 200:
        print('Error - {} is not available to access'.format(url))
        return
    
    # 將網頁回應的 HTML 傳入 BeautifulSoup 解析器, 方便我們根據標籤 (tag) 資訊去過濾尋找
    soup = BeautifulSoup(response.text)
    
    # 取得文章內容主體
    main_content = soup.find(id='main-content')
    
    # 假如文章有屬性資料 (meta), 我們在從屬性的區塊中爬出作者 (author), 文章標題 (title), 發文日期 (date)
    metas = main_content.select('div.article-metaline') #list
    author = ''
    title = ''
    date = ''
    if metas:
        if metas[0].select('span.article-meta-value')[0]:
            author = metas[0].select('span.article-meta-value')[0].string
        if metas[1].select('span.article-meta-value')[0]:
            title = metas[1].select('span.article-meta-value')[0].string
        if metas[2].select('span.article-meta-value')[0]:
            date = metas[2].select('span.article-meta-value')[0].string

        # 從 main_content 中移除 meta 資訊（author, title, date 與其他看板資訊）
        #
        # .extract() 方法可以參考官方文件
        #  - https://www.crummy.com/software/BeautifulSoup/bs4/doc/#extract
        for m in metas:
            m.extract()
        for m in main_content.select('div.article-metaline-right'):
            m.extract()
    
    # 取得留言區主體
    pushes = main_content.find_all('div', class_='push')
    for p in pushes:
        p.extract()
    
    # 假如文章中有包含「※ 發信站: 批踢踢實業坊(ptt.cc), 來自: xxx.xxx.xxx.xxx」的樣式
    # 透過 regular expression 取得 IP
    # 因為字串中包含特殊符號跟中文, 這邊建議使用 unicode 的型式 u'...'
    try:
        ip = main_content.find(text=re.compile(u'※ 發信站:'))
        ip = re.search('[0-9]*\.[0-9]*\.[0-9]*\.[0-9]*', ip).group()
    except Exception as e:
        ip = ''
    
    # 移除文章主體中 '※ 發信站:', '◆ From:', 空行及多餘空白 (※ = u'\u203b', ◆ = u'\u25c6')
    # 保留英數字, 中文及中文標點, 網址, 部分特殊符號
    #
    # 透過 .stripped_strings 的方式可以快速移除多餘空白並取出文字, 可參考官方文件 
    #  - https://www.crummy.com/software/BeautifulSoup/bs4/doc/#strings-and-stripped-strings
    filtered = []
    for v in main_content.stripped_strings:
        # 假如字串開頭不是特殊符號或是以 '--' 開頭的, 我們都保留其文字
        if v[0] not in [u'※', u'◆'] and v[:2] not in [u'--']:
            filtered.append(v)

    # 定義一些特殊符號與全形符號的過濾器
    expr = re.compile(u'[^一-龥。；，：“”（）、？《》\s\w:/-_.?~%()]')
    for i in range(len(filtered)):
        filtered[i] = re.sub(expr, '', filtered[i])
    
    # 移除空白字串, 組合過濾後的文字即為文章本文 (content)
    filtered = [i for i in filtered if i]
    content = ' '.join(filtered)
    
    # 處理留言區
    # p 計算推文數量
    # b 計算噓文數量
    # n 計算箭頭數量
    p, b, n = 0, 0, 0
    messages = []
    for push in pushes:
        # 假如留言段落沒有 push-tag 就跳過
        if not push.find('span', 'push-tag'):
            continue
        
        # 過濾額外空白與換行符號
        # push_tag 判斷是推文, 箭頭還是噓文
        # push_userid 判斷留言的人是誰
        # push_content 判斷留言內容
        # push_ipdatetime 判斷留言日期時間
        push_tag = push.find('span', 'push-tag').string.strip(' \t\n\r')
        push_userid = push.find('span', 'push-userid').string.strip(' \t\n\r')
        push_content = push.find('span', 'push-content').strings
        push_content = ' '.join(push_content)[1:].strip(' \t\n\r')
        push_ipdatetime = push.find('span', 'push-ipdatetime').string.strip(' \t\n\r')

        # 整理打包留言的資訊, 並統計推噓文數量
        messages.append({
            'push_tag': push_tag,
            'push_userid': push_userid,
            'push_content': push_content,
            'push_ipdatetime': push_ipdatetime})
        if push_tag == u'推':
            p += 1
        elif push_tag == u'噓':
            b += 1
        else:
            n += 1
    
    # 統計推噓文
    # count 為推噓文相抵看這篇文章推文還是噓文比較多
    # all 為總共留言數量 
    message_count = {'all': p+b+n, 'count': p-b, 'push': p, 'boo': b, 'neutral': n}
    
    # 整理文章資訊
    data = {
        'url': url,
        'article_author': author,
        'article_title': title,
        'article_date': date,
        'article_content': content,
        'ip': ip,
        'message_count': message_count,
        'messages': messages
    }
    return data

## 範例重點

分析列表取得要爬蟲的網址

In [17]:
# 對文章列表送出請求並取得列表主體
resp = requests.get(PTT_URL, cookies={'over18': '1'})
soup = BeautifulSoup(resp.text)
main_list = soup.find('div', class_='bbs-screen')
all_data = []

# 依序檢查文章列表中的 tag, 遇到分隔線就結束, 忽略這之後的文章
for div in main_list.findChildren('div', recursive=False):
    class_name = div.attrs['class']  #['search-bar']['r-ent']['r-ent']...
    
    # 遇到分隔線要處理的情況
    if class_name and 'r-list-sep' in class_name:
        print('Reach the last article')
        break
    
    # 遇到目標文章
    if class_name and 'r-ent' in class_name:
        div_title = div.find('div', class_='title')
        a_title = div_title.find('a', href=True)
        if a_title:
            article_URL = urljoin(PTT_URL, a_title['href'])
        else:
            article_URL = None
            a_title = '<a>本文已刪除</a>'
        article_title = a_title.text
        print('Parse {} - {}'.format(article_title, article_URL))
        
        # 呼叫上面寫好的 function 來對文章進行爬蟲
        if article_URL:
            parse_data = crawl_article(article_URL) # 返回單一文章資訊的字典
        
        # 將爬完的資料儲存
        all_data.append(parse_data)


Parse Re: [問卦] 幹 伯朗咖啡的口味太多了吧== - https://www.ptt.cc/bbs/Gossiping/M.1577519273.A.9B9.html
Parse [問卦] Minecraft產業鏈到底有多大???? - https://www.ptt.cc/bbs/Gossiping/M.1577519337.A.03F.html
Parse Re: [新聞] 鄭宏輝遭轟用「中國台灣」名義經商！　「 - https://www.ptt.cc/bbs/Gossiping/M.1577519346.A.569.html
Parse [問卦] 有沒有義消的八卦？ - https://www.ptt.cc/bbs/Gossiping/M.1577519384.A.25B.html
Parse [新聞] 夏威夷觀光直升機墜毀 尋獲6遺體、1人下落不明 - https://www.ptt.cc/bbs/Gossiping/M.1577519602.A.DDE.html
Parse Re: [爆卦] 館長要辦遊行啦 - https://www.ptt.cc/bbs/Gossiping/M.1577519711.A.A68.html
Parse [問卦] 求發財! 飛行夾克有啥大廠代工? - https://www.ptt.cc/bbs/Gossiping/M.1577519868.A.F27.html
Parse [問卦] 風雲起 山河動 預備 唱 - https://www.ptt.cc/bbs/Gossiping/M.1577520100.A.7A1.html
Parse Re: [問卦] 勞退提撥根本智障！ - https://www.ptt.cc/bbs/Gossiping/M.1577520115.A.02D.html
Parse [問卦] 跟朋友一起看片都去哪？ - https://www.ptt.cc/bbs/Gossiping/M.1577520315.A.9A4.html
Parse [問卦] 綁過小妹大腿的健身環可以賣多少？ - https://www.ptt.cc/bbs/Gossiping/M.1577520319.A.73E.html
Parse [問卦] 女醫森是不是都很棒？ - https://www.ptt.cc/bb

In [19]:
# 將爬完的資訊存成 json 檔案
with open('../Data/parse_data.json', 'w+', encoding='utf-8') as f:
    json.dump(all_data, f, ensure_ascii=False, indent=4)

以下為理解過程:

In [6]:
response = requests.get('https://www.ptt.cc/bbs/Gossiping/M.1577501860.A.D2F.html', cookies={'over18': '1'})
    
# 假設網頁回應不是 200 OK 的話, 我們視為傳送請求失敗
if response.status_code != 200:
    print('Error - {} is not available to access'.format(url))

# 將網頁回應的 HTML 傳入 BeautifulSoup 解析器, 方便我們根據標籤 (tag) 資訊去過濾尋找
soup = BeautifulSoup(response.text)

# 取得文章內容主體
main_content = soup.find(id='main-content')

# 假如文章有屬性資料 (meta), 我們在從屬性的區塊中爬出作者 (author), 文章標題 (title), 發文日期 (date)
metas = main_content.select('div.article-metaline') #list
author = ''
title = ''
date = ''
metas

[<div class="article-metaline"><span class="article-meta-tag">作者</span><span class="article-meta-value">joe911joeop (喬)</span></div>,
 <div class="article-metaline"><span class="article-meta-tag">標題</span><span class="article-meta-value">[問卦] 可能我的瘋狂  暫時不得到原諒？</span></div>,
 <div class="article-metaline"><span class="article-meta-tag">時間</span><span class="article-meta-value">Sat Dec 28 10:57:38 2019</span></div>]

In [7]:
print(main_content)
print('---------------------------')
if metas:
    if metas[0].select('span.article-meta-value')[0]:
        author = metas[0].select('span.article-meta-value')[0].string
    if metas[1].select('span.article-meta-value')[0]:
        title = metas[1].select('span.article-meta-value')[0].string
    if metas[2].select('span.article-meta-value')[0]:
        date = metas[2].select('span.article-meta-value')[0].string

    # 從 main_content 中移除 meta 資訊（author, title, date 與其他看板資訊）
    #
    # .extract() 方法可以參考官方文件
    #  - https://www.crummy.com/software/BeautifulSoup/bs4/doc/#extract
    for m in metas:
        m.extract()
    for m in main_content.select('div.article-metaline-right'):
        m.extract()
print(main_content)

<div class="bbs-screen bbs-content" id="main-content"><div class="article-metaline"><span class="article-meta-tag">作者</span><span class="article-meta-value">joe911joeop (喬)</span></div><div class="article-metaline-right"><span class="article-meta-tag">看板</span><span class="article-meta-value">Gossiping</span></div><div class="article-metaline"><span class="article-meta-tag">標題</span><span class="article-meta-value">[問卦] 可能我的瘋狂  暫時不得到原諒？</span></div><div class="article-metaline"><span class="article-meta-tag">時間</span><span class="article-meta-value">Sat Dec 28 10:57:38 2019</span></div>
可是我知道啊  可是我明白啊

是我的執著搏來  在你面前歌唱


吶，有沒有十年一刻的八卦呀？

--
<span class="f2">※ 發信站: 批踢踢實業坊(ptt.cc), 來自: 223.136.193.168 (臺灣)
</span><span class="f2">※ 文章網址: <a href="https://www.ptt.cc/bbs/Gossiping/M.1577501860.A.D2F.html" rel="nofollow" target="_blank">https://www.ptt.cc/bbs/Gossiping/M.1577501860.A.D2F.html</a>
</span><div class="push"><span class="hl push-tag">推 </span><span class="f3 hl push-userid">johnn

In [8]:
# 取得留言區主體，然後也將推文拿掉
pushes = main_content.find_all('div', class_='push')
for p in pushes:
    p.extract()
print(main_content)
print('~~~~~~~~')
# 假如文章中有包含「※ 發信站: 批踢踢實業坊(ptt.cc), 來自: xxx.xxx.xxx.xxx」的樣式
# 透過 regular expression 取得 IP
# 因為字串中包含特殊符號跟中文, 這邊建議使用 unicode 的型式 u'...'
try:
    ip = main_content.find(text=re.compile(u'※ 發信站:'))
    print('一',ip)
    ip = re.search('[0-9]*\.[0-9]*\.[0-9]*\.[0-9]*', ip).group()
    print('二',ip)
except Exception as e:
    ip = ''


<div class="bbs-screen bbs-content" id="main-content">
可是我知道啊  可是我明白啊

是我的執著搏來  在你面前歌唱


吶，有沒有十年一刻的八卦呀？

--
<span class="f2">※ 發信站: 批踢踢實業坊(ptt.cc), 來自: 223.136.193.168 (臺灣)
</span><span class="f2">※ 文章網址: <a href="https://www.ptt.cc/bbs/Gossiping/M.1577501860.A.D2F.html" rel="nofollow" target="_blank">https://www.ptt.cc/bbs/Gossiping/M.1577501860.A.D2F.html</a>
</span></div>
~~~~~~~~
一 ※ 發信站: 批踢踢實業坊(ptt.cc), 來自: 223.136.193.168 (臺灣)

二 223.136.193.168


In [10]:
# 移除文章主體中 '※ 發信站:', '◆ From:', 空行及多餘空白 (※ = u'\u203b', ◆ = u'\u25c6')
# 保留英數字, 中文及中文標點, 網址, 部分特殊符號
#
# 透過 .stripped_strings 的方式可以快速移除多餘空白並取出文字, 可參考官方文件 
#  - https://www.crummy.com/software/BeautifulSoup/bs4/doc/#strings-and-stripped-strings
filtered = []
for v in main_content.stripped_strings:
    print(v)
    # 假如字串開頭不是特殊符號或是以 '--' 開頭的, 我們都保留其文字
    if v[0] not in [u'※', u'◆'] and v[:2] not in [u'--']:
        filtered.append(v)
print('~~~')
print(filtered)

可是我知道啊  可是我明白啊

是我的執著搏來  在你面前歌唱


吶，有沒有十年一刻的八卦呀？

--
※ 發信站: 批踢踢實業坊(ptt.cc), 來自: 223.136.193.168 (臺灣)
※ 文章網址:
https://www.ptt.cc/bbs/Gossiping/M.1577501860.A.D2F.html
~~~
['可是我知道啊  可是我明白啊\n\n是我的執著搏來  在你面前歌唱\n\n\n吶，有沒有十年一刻的八卦呀？\n\n--', 'https://www.ptt.cc/bbs/Gossiping/M.1577501860.A.D2F.html']


In [11]:
# 定義一些特殊符號與全形符號的過濾器
expr = re.compile(u'[^一-龥。；，：“”（）、？《》\s\w:/-_.?~%()]')
for i in range(len(filtered)):
    filtered[i] = re.sub(expr, '', filtered[i])
print(filtered)

['可是我知道啊  可是我明白啊\n\n是我的執著搏來  在你面前歌唱\n\n\n吶，有沒有十年一刻的八卦呀？\n\n', 'https://www.ptt.cc/bbs/Gossiping/M.1577501860.A.D2F.html']


In [12]:
# 移除空白字串, 組合過濾後的文字即為文章本文 (content)
filtered = [i for i in filtered if i]
content = ' '.join(filtered)
print(filtered)

['可是我知道啊  可是我明白啊\n\n是我的執著搏來  在你面前歌唱\n\n\n吶，有沒有十年一刻的八卦呀？\n\n', 'https://www.ptt.cc/bbs/Gossiping/M.1577501860.A.D2F.html']


In [13]:
print(content)

可是我知道啊  可是我明白啊

是我的執著搏來  在你面前歌唱


吶，有沒有十年一刻的八卦呀？

 https://www.ptt.cc/bbs/Gossiping/M.1577501860.A.D2F.html
